<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/Resume_GPT_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Version of GPT Resume Seach Tool

## Installs and Imports

In [ ]:
!python3 -m pip install --upgrade langchain faiss chroma openai tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.9/849.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 5

In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.9 MB/s eta 0:00:00


In [ ]:
!pip install chroma

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7096 sha256=9e6c8fb9e2bbca11f46954fe988b1a0864229b8e013485adacc4cad16fd71a64
  Stored in directory: /root/.cache/pip/wheels/58/74/75/a6ab7999ae473ecbe819bc5cae9ccb902429dd6c60795f5112
Successfully built chroma


In [ ]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [ ]:
import argparse, json, time, datetime, openai
from pathlib import Path

In [ ]:
def set_open_ai_key(env_path=None):
  #import json, os
  #from pathlib import Path
  try:
    with open(env_path, "r") as f:
        env_vars = json.load(f)
    os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    #os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
    openai.Model.list() #test a random command on the openai API
    return True
  except Exception as e:
    print(e)
  return False

## Setup

In [ ]:
# setup API key
openai_env_path, openai.api_key = None, None
cwd = Path.cwd()
# resume_path = cwd / "Resumes"
# resume_path.mkdir(exist_ok=True)

openai_env_path = cwd/ "drive/MyDrive/Colab Notebooks/openai.env"
set_open_ai_key(openai_env_path)

True

## Load and Parse Resume Books

In [ ]:
cwd = Path.cwd()
output_path = cwd / "drive/MyDrive/Colab Notebooks/Output"

In [ ]:
#Set up document loader for pdf resume books
embeddings = OpenAIEmbeddings()
resume_path1 = "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"
resume_path2 = "/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf"

In [ ]:
def load_resumes(path):
  loader = PyPDFLoader("example_data/layout-parser-paper.pdf")
  pages = loader.load_and_split()
  return pages

In [ ]:
loader = PyPDFLoader(resume_path1)

In [ ]:
pages = loader.load_and_split()

In [ ]:
# actual resumes start on page 2 of this pdf compilation
resumes = pages[2:]

## Use Vector Stores to create embeddings and preform similarity search

to do: figure out which vector store to use - FAISS vs Chroma

In [ ]:
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

In [ ]:
faiss_index = FAISS.from_documents(resumes, OpenAIEmbeddings())

In [ ]:
docs = faiss_index.similarity_search("strong in math", k=4)

In [ ]:
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

2: YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfolio
7: JING (KALO) QIAN(614) 372-3321 ■kaloqian@nyu.edu■linkedin.com/in/jing-kaloqianEDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep 2021-Dec 2022)●Coursework:time series models (e.g., ARIMA, GARCH,MEM, Kalman filter), machinelearning
11: ZIYUAN (ALICE) ZHAO
(734) 353-3065 ■zz2408@nyu.edu■linkedin.com/in/ziyuan-zhao
EDUCA TION
NEW YORK UNIVERSITY New York, NY
The Courant Institute of Mathematical Sciences
M.S. in Mathematics in Finance(expected May 2023)
●Coursework:OOP in Java, test-driven development ,Black-Scholes model, stochasti
10: DAOMING ZHANG  dz1173@nyu.edu | (201) 296-5557 | www.linkedin.com/in/daom1ngz  EDUC

## Call OpenAI to use GPT to answer questions based on Resumes

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
chain = load_qa_with_sources_chain(llm, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
context = "Construct a list of Name fields from the documents given. Remove all duplicates from the list"

In [ ]:
ask('who knows machine learning')

In [ ]:
db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)

# What was the restaurant the group was talking about called?
query = input("Enter query:")

# The Hungry Lobster
ans = qa({"query": query})

print(ans)

In [ ]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result